In [55]:
#Newer pickle versions are not compatible
!which python
import pandas as pd
print(pd.__version__)
import sys
print(sys.version)
import inspect
print(inspect.getfile(pd))

/gpfs/bbp.cscs.ch/data/project/proj84/csaba/git/cellden/bin/python
2.2.1
3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 17:35:02) [GCC 11.2.0]
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/git/cellden/lib/python3.12/site-packages/pandas/__init__.py


In [2]:
import numpy as np
import pandas as pd
import pickle
import nrrd
import multiprocessing as mp
import re, os, copy
from voxcell import RegionMap
import ast

import sys
sys.path.append('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/notebooks/scripts/')

from helper_functions import get_all_filenames, get_csv_filenames, extract_prefix_from_filenames, read_and_concat_csv_files_new, combine_rows_and_calculate_average, create_combined_dataframe

In [3]:
download_base = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/'
root_folder = f"{download_base}results/density_calculations/"

# Load estimated non-scaled densities

In [4]:
%%time 

# Get all regional density data
folder_path = f"{root_folder}csv/"
filenames = get_all_filenames(folder_path)
csv_filenames = get_csv_filenames(folder_path)
prefixes = extract_prefix_from_filenames(csv_filenames)
unique_prefixes = sorted(list(set(prefixes)))

#Create a dict of df, each containing a cell type's occurence in all regions and its densities in all regions
result_dataframes = read_and_concat_csv_files_new(csv_filenames, unique_prefixes, folder_path)
combined_result_dataframes = combine_rows_and_calculate_average(result_dataframes)
shuffled_combined_dataframes = create_combined_dataframe(combined_result_dataframes)

file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/parcellation_to_parcellation_term_membership_extend.csv'
parcellation_annotation = pd.read_csv(file)
print("number of memberships at all levels:",len(parcellation_annotation))
parcellation_annotation = parcellation_annotation[parcellation_annotation['parcellation_term_set_name'] == 'substructure'] 
print("number of memberships at substructure level:",len(parcellation_annotation))
volumes = parcellation_annotation[['parcellation_label', 'parcellation_index', 'parcellation_term_acronym', 'parcellation_term_set_name', 'parcellation_term_name', 'voxel_count', 'volume_mm3', 'label_numbers', 'cluster_as_filename']]
substructure_vol = volumes[volumes['parcellation_term_set_name'] == 'substructure']


number of memberships at all levels: 3489
number of memberships at substructure level: 737
CPU times: user 37.7 s, sys: 173 ms, total: 37.8 s
Wall time: 37.8 s


In [5]:
# Initialize a list to store keys with problematic values
keys_with_issues = []

# Iterate over the dictionary
for key, df in combined_result_dataframes.items():
    # Check for NaN values
    has_nan = df.isna().values.any()
    
    # Check for inf values
    has_inf = df.isin([np.inf, -np.inf]).values.any()
    
    # Check for negative values
    has_negative = (df < 0).values.any()
    
    # If any issues are found, append the key to the list
    if has_nan or has_inf or has_negative:
        keys_with_issues.append({
            "key": key,
            "has_nan": has_nan,
            "has_inf": has_inf,
            "has_negative": has_negative
        })

# Display the keys that contain issues
if keys_with_issues:
    print("The following DataFrames have issues:")
    for issue in keys_with_issues:
        print(f"Key: {issue['key']}, NaN: {issue['has_nan']}, Inf: {issue['has_inf']}, Negative: {issue['has_negative']}")
else:
    print("No issues found in combined_result_dataframes DataFrames.")

No issues found in combined_result_dataframes DataFrames.


In [6]:
#write PICKLE file with densities
# root_folder = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'
# Assuming 'combined_result_dataframes' is your dictionary of DataFrames
with open(f'{root_folder}non_scaled_densities_new.pickle', 'wb') as f:
    pickle.dump(combined_result_dataframes, f)

print(f"Pickle file saved here: {root_folder}")

Pickle file saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/


In [7]:
#Read CCFv3 annotation volumes (choose 1) for calculating total cell counts
# data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
# CCFv3, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3.nrrd')
# data_folder = "/gpfs/bbp.cscs.ch/data/project/proj62/csaba/atlas/bbp_prod_files/2022/"
# CCFv3, _ = nrrd.read(f'{data_folder}annotation_25.nrrd')
data_folder = "/gpfs/bbp.cscs.ch/project/proj84/piluso/share/general/warped_augmented_CCFv3/"
CCFv3, _ = nrrd.read(f'{data_folder}annotation_25_2022_CCFv3a.nrrd')
# CCFv3, _ = nrrd.read("/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/annotation_ccfv3_l23split_barrelsplit_validated.nrrd")

# Create float copy of the annotation voluem, update values in CCFv3 based on conditions
#CCFv3_copy = np.copy(CCFv3).astype('float64')

#hierarchy_json = '/gpfs/bbp.cscs.ch/home/veraszto/bbp_prod_files/1.json'
hierarchy_json = '/gpfs/bbp.cscs.ch/data/project/proj84/atlas_pipeline_runs/2024-05-15T22:44:26+02:00/hierarchy_ccfv3_l23split_barrelsplit.json'
region_map = RegionMap.load_json(hierarchy_json)


# Local (area by area) scaling when necessary
Scaling parameters are provided by Blue Brain in the next cell

In [8]:
# Create a dictionary from the expected data:
data = {
    "id": [391, 399, 407, 415, 431, 438, 446, 454, 471, 486, 495, 504, 718, 733, 1020],
    "acronym": [
        "CA1slm", "CA1so", "CA1sp", "CA1sr", "CA2slm", "CA2so", 
        "CA2sp", "CA2sr", "CA3slm", "CA3so", "CA3sp", "CA3sr", "VPL", "VPM", "PO"
    ],
    "Region": [
        "Field CA1, stratum lacunosum-moleculare", "Field CA1, stratum oriens",
        "Field CA1, stratum pyramidale", "Field CA1, stratum radiatum",
        "Field CA2, stratum lacunosum-moleculare", "Field CA2, stratum oriens",
        "Field CA2, stratum pyramidale", "Field CA2, stratum radiatum",
        "Field CA3, stratum lacunosum-moleculare", "Field CA3, stratum oriens",
        "Field CA3, stratum pyramidale", "Field CA3, stratum radiatum",
        "Ventral posterolateral nucleus of the thalamus", "Ventral posteromedial nucleus of the thalamus",
        "Posterior complex of the thalamus"
    ],
    "Expected Inhibitory Dorsal": [
        8420, 6480, 13520, 6480, 8420, 6480, 13520, 3240, 5780, 3170, 7560, 7890, "", "", ""
    ],
    "Expected Inhibitory Ventral": [
        8820, 10500, 10130, 10500, 8820, 10500, 10130, 4260, 7200, 5460, 10540, 6590, "", "", ""
    ],
    "Expected Neurons Dorsal": [
        "", "", 447500, "", "", "", 447500, "", "", "", "", "", "", "", ""
    ],
    "Expected Neurons Ventral": [
        "", "", 180500, "", "", "", 180500, "", "", "", "", "", "", "", ""
    ],
    "exc_mm3": [
        "", "", "", "", "", "", "", "", "", "", "", "", 55340.64, 67940.58, ""
    ],
    "inh_mm3": [
        8620, 8490, 11825, 8490, 8620, 8490, 11825, 3750, 6490, 4315, 9050, 7240, 2126.28, 2978.61, 161.08663
    ],
    "neurons_mm3": [
        "", "", 314000, "", "", "", 314000, "", "", "", 172400, "", 57466.92, 70919.19, ""
    ]
}

# Create a DataFrame from the dictionary
df_e = pd.DataFrame(data)


# New data to be added
new_data = {
    "id": [212, 220, 228, 236, 244, 477],
    "acronym": [
        "MOBglomerularlayer", "MOBgr", "MOBipl", "MOBmi", "MOBopl", "STR"
    ],
    "Region": [
        "Main olfactory bulb, glomerular layer", "Main olfactory bulb, granule layer",
        "Main olfactory bulb, inner plexiform layer", "Main olfactory bulb, mitral layer",
        "Main olfactory bulb, outer plexiform layer", "Striatum"
    ],
    "Expected Inhibitory Dorsal": ["", "", "", "", "", ""],
    "Expected Inhibitory Ventral": ["", "", "", "", "", ""],
    "Expected Neurons Dorsal": ["", "", "", "", "", ""],
    "Expected Neurons Ventral": ["", "", "", "", "", ""],
    "exc_mm3": ["", "", "", "", "", ""],
    "inh_mm3": ["", "", "", "", "", ""],
    "neurons_mm3": [
        630000, 710000, 150000, 350000, 80000, 78560
    ]
}
df_new = pd.DataFrame(new_data)

# Concatenate the new data with the existing data
df_expected = pd.concat([df_e, df_new], ignore_index=True)
df_expected.to_csv('/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/expected_data.csv', index=False)

#Add children to the regions which need to change to include non-leaf-region changes:
file = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/df_hierarchy_ccfv3_l23split_barrelsplit.csv'
hierarchy = pd.read_csv(file, index_col=0)

# Merge df_expected with hierarchy on 'id' to get the 'children' column
df_expected = df_expected.merge(hierarchy[['id', 'children', 'acr_list']], on='id', how='left')

import ast
df_expected.loc[: , 'children'] = df_expected.loc[: , 'children'].apply(ast.literal_eval)

# Add new columns to df_expected
df_expected['exc_mm3_ratio'] = np.nan
df_expected['inh_mm3_ratio'] = np.nan
df_expected['neurons_mm3_ratio'] = np.nan

## Total nrrd creation
We can separate cell types into larger (hierarchical groups)

In [9]:
%%time 

meta_path = "/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/metadata/WMB-10X/20231215/views/cell_metadata_with_cluster_annotation.csv"
columns_to_read = ['class', 'subclass', 'cluster']
#metadata = pd.read_csv(meta_path, dtype={'cell_label':str}, low_memory=False)
metadata = pd.read_csv(meta_path, usecols=columns_to_read, )

n_classes = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
       '04 DG-IMN Glut', '05 OB-IMN GABA', '06 CTX-CGE GABA',
       '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
       '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '13 CNU-HYa Glut',
       '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
       '18 TH Glut', '19 MB Glut', '20 MB GABA', '21 MB Dopa',
       '22 MB-HB Sero', '23 P Glut', '24 MY Glut', '25 Pineal Glut',
       '26 P GABA', '27 MY GABA', '28 CB GABA', '29 CB Glut',]

nn_classes = ['30 Astro-Epen', '31 OPC-Oligo', '32 OEC', '33 Vascular',
       '34 Immune']

exc = ['01 IT-ET Glut', '02 NP-CT-L6b Glut', '03 OB-CR Glut',
      '04 DG-IMN Glut', '13 CNU-HYa Glut', '14 HY Glut', '15 HY Gnrh1 Glut', '16 HY MM Glut', '17 MH-LH Glut',
      '18 TH Glut', '19 MB Glut', '23 P Glut', '24 MY Glut', '25 Pineal Glut', '29 CB Glut',]
inh = ['05 OB-IMN GABA', '06 CTX-CGE GABA', '07 CTX-MGE GABA', '08 CNU-MGE GABA', '09 CNU-LGE GABA',
      '10 LSX GABA', '11 CNU-HYa GABA', '12 HY GABA', '20 MB GABA', '26 P GABA', '27 MY GABA', '28 CB GABA', ]
other = ['21 MB Dopa', '22 MB-HB Sero', ]
exci_inhib_sum = exc + inh

astrotypes = ['5206 Bergmann NN_1', '5207 Astro-CB NN_1', '5208 Astro-NT NN_1',
       '5209 Astro-NT NN_1', '5210 Astro-NT NN_1', '5211 Astro-NT NN_1',
       '5212 Astro-NT NN_1', '5213 Astro-NT NN_1', '5214 Astro-NT NN_2',
       '5215 Astro-NT NN_2', '5216 Astro-NT NN_2', '5217 Astro-NT NN_2',
       '5218 Astro-TE NN_1', '5219 Astro-TE NN_1', '5220 Astro-TE NN_1',
       '5221 Astro-TE NN_1', '5222 Astro-TE NN_2', '5223 Astro-TE NN_2',
       '5224 Astro-TE NN_3', '5225 Astro-TE NN_3', '5226 Astro-TE NN_3',
       '5227 Astro-TE NN_3', '5228 Astro-TE NN_4', '5229 Astro-TE NN_5',
       '5230 Astro-TE NN_5', '5231 Astro-OLF NN_1', '5232 Astro-OLF NN_1',
       '5233 Astro-OLF NN_2', '5234 Astro-OLF NN_2',
       '5235 Astro-OLF NN_3', '5236 Astro-OLF NN_3',]

microglia = ['5312 Microglia NN_1']

oligos = [ '5266 OPC NN_1', '5267 OPC NN_1',
       '5268 OPC NN_1', '5269 OPC NN_1', '5270 OPC NN_1', '5271 OPC NN_2',
       '5272 COP NN_1', '5273 COP NN_1', '5274 COP NN_1', '5275 COP NN_1',
       '5276 COP NN_1', '5277 COP NN_1', '5278 NFOL NN_2',
       '5279 NFOL NN_2', '5280 NFOL NN_2', '5281 NFOL NN_2',
       '5282 MFOL NN_3', '5283 MFOL NN_3', '5284 MOL NN_4',
       '5285 MOL NN_4', '5286 MOL NN_4', '5287 MOL NN_4', '5288 MOL NN_4',]

glia = astrotypes + microglia + oligos

neurontypes = np.unique(metadata[metadata['class'].isin(n_classes)]['cluster'].values)
nonneurontypes = np.unique(metadata[metadata['class'].isin(nn_classes)]['cluster'].values)
exctypes = np.unique(metadata[metadata['class'].isin(exc)]['cluster'].values)
inhtypes = np.unique(metadata[metadata['class'].isin(inh)]['cluster'].values)
othertypes = np.unique(metadata[metadata['class'].isin(other)]['cluster'].values)
exci_inhib_sum = np.unique(metadata[metadata['class'].isin(exci_inhib_sum)]['cluster'].values)
celltypes = np.unique(metadata['cluster'].values)

CPU times: user 16.2 s, sys: 403 ms, total: 16.6 s
Wall time: 16.7 s


In [10]:
#Helper function to create total cell count values for a 3D brain

# import re
# import pandas as pd
# import numpy as np

def nrrd_for_validation(df, parcellation_annotation, CCFv3):
    all_ids_for_df = []
    df_comb = pd.DataFrame()

    for regionname in df.index.values[0:]:
        density = df.loc[regionname, 'density_mm3']
        #annotation_id_info = substructures[substructures['cluster_as_filename'] == regionname]
        annotation_id_info = parcellation_annotation[parcellation_annotation['cluster_as_filename'] == regionname]

        Annotation2020ids = [int(re.search(r'\d+$', s).group()) for s in annotation_id_info['parcellation_label'].values]
        df_sub = pd.DataFrame({'density': density}, index=Annotation2020ids)
        df_comb = pd.concat([df_comb, df_sub])
        all_ids_for_df.append(Annotation2020ids)

    all_ids_for_df = [value for sublist in all_ids_for_df for value in sublist]
    all_ids_for_df.append(0)
    #Place to put extra regions not part of Allen's Parcellation annotation

    outside = 0
    outsideid = [0]
    df_sub = pd.DataFrame({'density': outside}, index=outsideid)
    df_comb = pd.concat([df_comb, df_sub])

    CCFv3_copy = CCFv3.copy()

    # Expression is 0 in those regions where we don't have any info:
    CCFv3_copy[~np.isin(CCFv3_copy, all_ids_for_df)] = 0.0 

    # Expression is non-zero in these leaf region(s)
    for index, row in df_comb.iterrows():
        density_value = row['density']
        region_id = index
        CCFv3_copy[np.isin(CCFv3, region_id)] = density_value

    #Create outside of the brain as 0
    CCFv3_copy[np.isin(CCFv3, int(0))] = 0

    return CCFv3_copy

In [11]:
%%time

#from multiprocessing import Pool

def process_type(types, file_name):
    # Filter DataFrames based on types
    filtered_dataframes = {key: value for key, value in shuffled_combined_dataframes.items() if key in types}
    
    # Combine filtered DataFrames
    combined_df = pd.concat(filtered_dataframes.values())
    
    # Sum the combined DataFrame by index
    summed_df = combined_df.groupby(combined_df.index).sum()
    
    # Validate result
    result = nrrd_for_validation(summed_df, parcellation_annotation, CCFv3)
    
    # Clean up
    del combined_df, summed_df, filtered_dataframes

    return (result, file_name)

def main():
    # Define the parameters for each process
    tasks = [
        (neurontypes, "non_scaled_total_neuron_densities"),
        (exctypes, "non_scaled_total_excitatory_densities"),
        (inhtypes, "non_scaled_total_inhibitory_densities"),
        (astrotypes, "non_scaled_total_astrotypes_densities"),
        (microglia, "non_scaled_total_microglia_densities"),
        (oligos, "non_scaled_total_oligocyte_densities"),
        (glia, "non_scaled_total_glia_densities"),
        (exci_inhib_sum, "non_scaled_total_excinh_densities"),
        (celltypes, "non_scaled_total_celltypes_densities"),
        (nonneurontypes, "non_scaled_total_nonneuron_densities")
    ]
    
    # Create a multiprocessing Pool
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.starmap(process_type, tasks)
    
    # Sequentially write the .nrrd files to avoid concurrent writes
    for result, file_name in results:
        nrrd.write(f"{root_folder}{file_name}.nrrd", result)
        print(f"{root_folder}{file_name}.nrrd")
        
if __name__ == "__main__":
    main()

/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/non_scaled_total_neuron_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/non_scaled_total_excitatory_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/non_scaled_total_inhibitory_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/non_scaled_total_astrotypes_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/non_scaled_total_microglia_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/non_scaled_total_oligocyte_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/non_scaled_total_glia_densities.nrrd
/gpfs/bbp.cscs.ch/

In [12]:
#Add scaling parameter to df_expected too by loading scaled total density files and search for corresponding values (computed earlier)

#NEURON
filename = "non_scaled_total_neuron_densities.nrrd"
full_path = os.path.join(root_folder, filename)
if os.path.isfile(full_path):
    neuron_total, _ = nrrd.read(full_path)
    print(filename, " was opened.")

#EXC
filename = "non_scaled_total_excitatory_densities.nrrd"
full_path = os.path.join(root_folder, filename)
if os.path.isfile(full_path):
    exc_total, _ = nrrd.read(full_path)
    print(filename, " was opened.")
    
#INH        
filename = "non_scaled_total_inhibitory_densities.nrrd"
full_path = os.path.join(root_folder, filename)
if os.path.isfile(full_path):
    inh_total, _ = nrrd.read(full_path)
    print(filename, " was opened.")


non_scaled_total_neuron_densities.nrrd  was opened.
non_scaled_total_excitatory_densities.nrrd  was opened.
non_scaled_total_inhibitory_densities.nrrd  was opened.


In [13]:
%%time 

'''We divide by: mean expected / transcriptomic mean densities..'''

# Arrays to process
arrays = {
    'neurons_mm3': neuron_total,
    'inh_mm3': inh_total,
    'exc_mm3': exc_total,
}

# Worker function to process each row
def process_row(row):
    id_ = row['id'] 
    acr = row['acronym']
    name = row['Region']
    ids = row['children']
    ids.append(id_) #Make sure you add the original id_ to the children
    
    
    # Create a mask for the current id list
    mask = np.isin(CCFv3, [ids])
    
    # Calculate mean/median values (can place NaN or None in the regions not present in CCfV3)
    # mean_values = {name: arr[mask].mean() if arr[mask].size > 
    #                0 else np.nan for name, arr in arrays.items()}
    mean_values = {name: arr[mask].mean() if arr[mask].size > 
                   0 else None for name, arr in arrays.items()}
    # mean_values = {name: np.median(arr[mask]) if arr[mask].size > 
    #                0 else None for name, arr in arrays.items()}

    # Update row with ratio values
    if mean_values['exc_mm3'] is not np.nan and isinstance(row['exc_mm3'], (int, float)):
        row['exc_mm3_ratio'] = row['exc_mm3'] / mean_values['exc_mm3']
    if mean_values['inh_mm3'] is not np.nan and isinstance(row['inh_mm3'], (int, float)):
        row['inh_mm3_ratio'] = row['inh_mm3'] / mean_values['inh_mm3']
    if mean_values['neurons_mm3'] is not np.nan and isinstance(row['neurons_mm3'], (int, float)):
        row['neurons_mm3_ratio'] = row['neurons_mm3'] / mean_values['neurons_mm3']
    
    return row
    
    
# Use multiprocessing Pool to process rows in parallel
if __name__ == '__main__':
    with mp.Pool(processes=mp.cpu_count()) as pool:
        new_data = pool.map(process_row, [row for _, row in df_expected.iterrows()])
    
    # Convert the list of rows back to a DataFrame
    df_expected = pd.DataFrame(new_data)



CPU times: user 58.9 ms, sys: 1.03 s, total: 1.09 s
Wall time: 5.38 s


In [14]:
#import ast 
#Note that we will scale the scaled data: scaled_combined_result_dataframes
combined_result_dataframes_copy = copy.deepcopy(combined_result_dataframes)


for reg in df_expected['acronym'].values:
    concatenated_result = np.array(df_expected['acronym'].values) 
    if reg not in list(combined_result_dataframes_copy.keys()):
        print(reg, " is not a leaf region recognised by transcriptomics data, it will be split to its leaves")
        #leaves_list = df_expected[df_expected['acronym'] == reg]['children'].iat[0]
        acr_list_str = df_expected[df_expected['acronym'] == reg]['acr_list'].iat[0]
        acr_list = ast.literal_eval(acr_list_str)

        # Concatenate the array and the list to cover the full area where secondary scaling takes effect
        # We combine the leaf regions with the list of regions non leaf regions
        concatenated_result = np.concatenate((concatenated_result, acr_list))
        print(len(acr_list), len(df_expected['acronym'].values), len(concatenated_result))

STR  is not a leaf region recognised by transcriptomics data, it will be split to its leaves
30 21 51


In [15]:
#Total number of transplants
len(concatenated_result)

51

In [16]:
df_expected.tail(2)

,id,acronym,Region,Expected Inhibitory Dorsal,Expected Inhibitory Ventral,Expected Neurons Dorsal,Expected Neurons Ventral,exc_mm3,inh_mm3,neurons_mm3,children,acr_list,exc_mm3_ratio,inh_mm3_ratio,neurons_mm3_ratio
19,244,MOBopl,"Main olfactory bulb, outer plexiform layer",,,,,,,80000,"[244, 244]",['MOBopl'],NaN,NaN,0.399253
20,477,STR,Striatum,,,,,,,78560,"[3034756217, 672, 56, 998, 549009199, 310, 333...","['STR_O', 'CP', 'ACB', 'FS', 'LSS', 'SF', 'SH'...",NaN,NaN,0.496897


In [17]:
%%time

# del scaled_combined_result_dataframes
# We create the dict of dfs anew!
scaled_combined_result_dataframes = {}
keys_to_print = []

#Do the 2ndary scaling: we check every transcriptomic region and scale their cell types if needed: 
for key, df in combined_result_dataframes_copy.items():
    if key in concatenated_result:
        keys_to_print.append(key)
        # We find the row where this region is listed (can be a substring if the row is not a leaf region)
        matches = df_expected.map(lambda x: key in str(x))
        rows_with_substring = df_expected[matches.any(axis=1)]
        # If the region has a non-NAN value in the ratios, we multiply that type of cells with the ratio
        if not rows_with_substring.empty and pd.notna(rows_with_substring['exc_mm3_ratio'].iloc[0]):
            mask = df.index.isin(exctypes)
            df.loc[mask, 'density_mm3'] *= rows_with_substring['exc_mm3_ratio'].iloc[0]
            scaled_combined_result_dataframes[key] = df
        #break
        if not rows_with_substring.empty and pd.notna(rows_with_substring['inh_mm3_ratio'].iloc[0]):
            mask = df.index.isin(inhtypes)
            df.loc[mask, 'density_mm3'] *= rows_with_substring['inh_mm3_ratio'].iloc[0]
            scaled_combined_result_dataframes[key] = df

        if not rows_with_substring.empty and pd.notna(rows_with_substring['neurons_mm3_ratio'].iloc[0]):
            mask = df.index.isin(neurontypes)
            df.loc[mask, 'density_mm3'] *= rows_with_substring['neurons_mm3_ratio'].iloc[0]
            scaled_combined_result_dataframes[key] = df
            #break
        #print("Deleting data for " + key)
        del key, df        
    else:
       #In this case we don't have to scale
        scaled_combined_result_dataframes[key] = df
        del key, df

if keys_to_print:
    print(", ".join(keys_to_print) + " is a leaf region which will be transplanted.")

AAA, ACB, BA, CA1slm, CA1so, CA1sp, CA1sr, CA2slm, CA2so, CA2sp, CA2sr, CA3slm, CA3so, CA3sp, CA3sr, CEAc, CEAl, CEAm, CP, FS, IA, LSc, LSr, LSv, MOBglomerularlayer, MOBgr, MOBipl, MOBmi, MOBopl, PO, SF, SH, VPL, VPM is a leaf region which will be transplanted.
CPU times: user 92.7 ms, sys: 4.93 ms, total: 97.6 ms
Wall time: 96.3 ms


In [18]:
# Assuming 'scaled_combined_result_dataframes' is your dictionary of DataFrames
with open(f'{root_folder}scaled_densities_local.pickle', 'wb') as f:
    pickle.dump(scaled_combined_result_dataframes, f)

print(f"Saved here: {root_folder}")

Saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/


In [19]:
#del scaled_combined_result_dataframes

# Global scaling

This notebook starts after data was created from MERFISH slices and stored as csv and pickle files. We examine scaling individual cell groups based on literature: https://www.pnas.org/doi/full/10.1073/pnas.0604911103

Total cells (M): 108.69 ± 16.25

Total neurons (M): 70.89 ± 10.41

CCTX: 17.8 ± 3.4% of neurons (this is not the isocortex)

CRB: 59.0 ± 5.0% of neurons

"Cerebral cortex" 688 = "Cortical plate" 695 + "Cortical subplate" 703
"Cerebral nuclei is not part of the cer.ctx)
Cerebral cortex is not the Cerebellar cortex (the Cerebellar cortex is part of the Cerebellum)
From test_tutorial_cerebellum.ipynb we decided to only scale exc cells (ie granular layer) in the CRB.



In [20]:
#If we want we can load the previous step

file = os.path.join( root_folder, 'scaled_densities_local.pickle' )
#file = os.path.join( root_folder, 'non_scaled_densities_new.pickle' )

#Load region id volumes from volume_calc_from_template.ipynb
with open(file, 'rb') as pickle_file:
    scaled_combined_result_dataframes = pickle.load(pickle_file)
print("Loaded pickle file.")

Loaded pickle file.


In [21]:
#in this cell we just give ground truth cell counts from the paper
total_cells = 108.69 * 1_000_000
total_neurons = 70.89 * 1_000_000
total_ctx = total_neurons * 17.8 / 100 #17.8% of the total neurons
total_crb = total_neurons * 59.0 / 100 #59% of the total neurons
total_rest = total_neurons * ( 100 - 59.0 -17.8 )/ 100
print("Neurons only: ")
print("CTX: " '{:,.0f}'.format(total_ctx,), "CRB: " '{:,.0f}'.format(total_crb),"REST: " '{:,.0f}'.format(total_rest))

Neurons only: 
CTX: 12,618,420 CRB: 41,825,100 REST: 16,446,480


In [22]:
#Next we have to convert densities to cell counts for the scaling. This way we can measure the scaling ratio between the current densities and the desired ones. For this we need voxel size

# Edge length of the voxel in millimeters
edge_length_mm = 25.0 / 1000.0  # Convert micrometers to millimeters

# Calculate the volume of the voxel in cubic millimeters
volume25_mm3 = round(edge_length_mm ** 3, 10)

print("Volume of the voxel in mm^3:", volume25_mm3)

# Edge length of the voxel in millimeters
edge_length_mm = 10.0 / 1000.0  # Convert micrometers to millimeters

# Calculate the volume of the voxel in cubic millimeters
volume10_mm3 = round(edge_length_mm ** 3, 10)

print("Volume of the voxel in mm^3:", volume10_mm3)


Volume of the voxel in mm^3: 1.5625e-05
Volume of the voxel in mm^3: 1e-06


Next, we convert densities to total cell number / region

In [23]:
%%time 
scaled_total_cells_local = {}

if CCFv3.shape == (528, 320, 456):
    print("annotation vol is 25 um resolution")
    for region_name, df in scaled_combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['cluster_as_filename'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        volume = np.count_nonzero(CCFv3 == id_) * volume25_mm3
        #print(region_name, label, id_, f"{volume:.6f}")
        # Calculate total cells by multiplying density by volume
        scaled_total_cells_local[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})

elif CCFv3.shape == (566, 320, 456):
    print("annotation vol is 25 um resolution but it is larger than the Allen Institute's CCFv3")
    for region_name, df in scaled_combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['cluster_as_filename'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        volume = np.count_nonzero(CCFv3 == id_) * volume25_mm3
        #print(region_name, label, id_, f"{volume:.6f}")
        # Calculate total cells by multiplying density by volume
        scaled_total_cells_local[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})

elif CCFv3.shape == (1320, 800, 1140) or CCFv3.shape == (1140, 800, 1320):
    print("*** annotation vol is 10 um resolution *** ")
    for region_name, df in scaled_combined_result_dataframes.items():
        # Get the corresponding volume for the region
        label = volumes.loc[volumes['cluster_as_filename'] == region_name, 'parcellation_label'].values[0]
        id_ = int(label.rsplit('-', 1)[-1])
        #id_ = region_map.find(region_name, attr='name', ignore_case=True, with_descendants=False)
        #print(region_name, label, id_)
        
        if CCFv3.shape == (1320, 800, 1140):
            volume = np.count_nonzero(CCFv3 == id_) * volume10_mm3
        elif CCFv3.shape == (1140, 800, 1320):
            volume = np.count_nonzero(CCFv3 == id_) * volume10_mm3 
        # Calculate total cells by multiplying density by volume
        print(region_name, label, id_, f"{volume:.6f}")
        scaled_total_cells_local[region_name] = pd.DataFrame({'total_cellnumber': df['density_mm3'] * volume})
else:
    print("*** Annotation volume shape is not understood. *** ")

#We can save the file just in case. 

# Assuming 'scaled_total_cells_local' is your dictionary of DataFrames
with open(f'{root_folder}scaled_total_cells_local.pickle', 'wb') as f:
    pickle.dump(scaled_total_cells_local, f)

# # Assuming you skipped transplantation and only want to run global scaling 
# with open(f'{root_folder}scaled_globalonly_cells.pickle', 'wb') as f:
#     pickle.dump(scaled_total_cells_local, f)

print(f"Saved here: {root_folder}")

annotation vol is 25 um resolution but it is larger than the Allen Institute's CCFv3
Saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/
CPU times: user 40.4 s, sys: 11.3 ms, total: 40.4 s
Wall time: 40.4 s


In [24]:
#Now, we need to isolate specific regions from the others to scale them differently

isocortex = (
    region_map.find("Isocortex", attr="name", with_descendants=True)
    | region_map.find("Entorhinal area", attr="name", with_descendants=True)
    | region_map.find("Piriform area", attr="name", with_descendants=True)
)
cerebellum = region_map.find(
        "Cerebellum", attr="name", with_descendants=True
    ) | region_map.find("arbor vitae", attr="name", with_descendants=True)

fiber_tracts_ids = (
    region_map.find("fiber tracts", attr="name", with_descendants=True)
    | region_map.find("grooves", attr="name", with_descendants=True)
    | region_map.find("ventricular systems", attr="name", with_descendants=True)
)
hippocampus = (
    region_map.find("Hippocampal region", attr="name", with_descendants=True)
)
hippocampal_formation = (
    region_map.find("Hippocampal formation", attr="name", with_descendants=True)
)
thalamus = (
    region_map.find("Thalamus", attr="name", with_descendants=True)
)
striatum = (
    region_map.find("Striatum", attr="name", with_descendants=True)
)
VPL = (
    region_map.find("Ventral posterolateral nucleus of the thalamus", attr="name", with_descendants=True)
)
LGd = (
    region_map.find("Dorsal part of the lateral geniculate complex", attr="name", with_descendants=True)
)
VPM = (
    region_map.find("Ventral posteromedial nucleus of the thalamus", attr="name", with_descendants=True)
)
MOB = (
    region_map.find("Main olfactory bulb", attr="name", with_descendants=True)
)

#This cell will complement the two cells above
Hindbrain = (
    region_map.find("Hindbrain", attr="name", with_descendants=True)
)
Midbrain = (
    region_map.find("Midbrain", attr="name", with_descendants=True)
)
Interbrain = (
    region_map.find("Interbrain", attr="name", with_descendants=True)
)
Olfactoryareas = (
    region_map.find("Olfactory areas", attr="name", with_descendants=True)
)
Corticalsubplate = (
    region_map.find("Cortical subplate", attr="name", with_descendants=True)
)
Cerebralnuclei = (
    region_map.find("Cerebral nuclei", attr="name", with_descendants=True)
)
cerebral_cortex = (
    region_map.find("Cerebral cortex", attr="name", with_descendants=True)
)
wholebrain = (
    region_map.find("root", attr="name", with_descendants=True)
)

#This is the final dictionary of all region names and their region ids
areas = {
    'wholebrain': wholebrain, 'Cerebral cortex': cerebral_cortex,
    'isocortex': isocortex, 'cerebellum': cerebellum, 'fiber_tracts_ids': fiber_tracts_ids, 
    'hippocampus': hippocampus, 'hippocampal_formation':hippocampal_formation, 
    'thalamus': thalamus, 'striatum': striatum, 'VPL': VPL, 'LGd': LGd, 'VPM': VPM, 'MOB': MOB,
    'Olfactory areas': Olfactoryareas, 'Cortical subplate': Corticalsubplate, 
    'Cerebral nuclei': Cerebralnuclei, 'Interbrain': Interbrain , 'Midbrain': Midbrain, 
    'Interbrain': Interbrain
}

areas_min = {
    'wholebrain': wholebrain, 'Cerebral cortex': cerebral_cortex, 'cerebellum': cerebellum,
            }

In [25]:
#We have to filter unassigned regions since we don't know enough of them.

regions = list(scaled_total_cells_local.keys())
print(f'Regions _covered_ in the brain / total cells dict of df: {len(regions)}')
# Assuming 'my_list' is your list
if 'unassigned' in regions:
    print("Region was found, will remove: unassigned.")
    regions.remove('unassigned')
    print("For now we are removing the region unassigned as we cannot place it in the hierarchy.json system")
    
if 'brainunassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brainunassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system")

if 'brain-unassigned' in regions:
    print("Region was found, will remove brain-unassigned.")
    regions.remove('brain-unassigned')
    print("For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system") 
    
print(f"Remaining regions: {len(regions)}")
region_info = parcellation_annotation[parcellation_annotation['cluster_as_filename'].isin(regions)]
# region_info = parcellation_annotation[parcellation_annotation['parcellation_term_acronym'].isin(regions)]

region_info.shape

Regions _covered_ in the brain / total cells dict of df: 703
Region was found, will remove: unassigned.
For now we are removing the region unassigned as we cannot place it in the hierarchy.json system
Region was found, will remove brain-unassigned.
For now we are removing the region brain-unassigned as we cannot place it in the hierarchy.json system
Remaining regions: 701


(708, 19)

In [26]:
#Next we can calculate total cell numbers overall and in specific regions

scaled_total_cells_local_copy = copy.deepcopy(scaled_total_cells_local) 
ground_truth_cell_numbers = []

for key, df in scaled_total_cells_local_copy.items():
    scaled_total_cells_local_copy[key] = df[df.index.isin(neurontypes)]

for key, value in areas_min.items():
    
    keys_to_drop = ['unassigned', 'brain-unassigned']
    for region in regions:
        
        #filtered_data = region_info[(region_info['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        filtered_data = parcellation_annotation[(parcellation_annotation['cluster_as_filename'] == region) &(region_info['parcellation_term_set_name'] == 'substructure')]
        id_ = filtered_data['label_numbers'].iloc[0]
        
        if int(id_) in value:
            #print(region, id_, "is in the set")
            None
        else:
            #print(id_, "is not in the set")
            keys_to_drop.append(region)

        #break    

    # Create a new dictionary without the specified keys (all leaf regions and their cell types in 1 region)
    scaled_total_cells_local_filtered = {key: value for key, value in scaled_total_cells_local_copy.items() if key not in keys_to_drop}

    # Concatenate all DataFrames into a single DataFrame
    concatenated_df = pd.concat(scaled_total_cells_local_filtered.values())

    # Group by all columns except the 'total_cells' column and sum the values
    summed_df = concatenated_df.groupby(concatenated_df.index).sum()
    print(f"The brain area {key} has {summed_df.shape[0]} different cell types")
    #print("Total sum of cells:", '{:,.0f}'.format(summed_df['density_mm3'].sum()))
    print("Total sum of neurons:", '{:,.0f}'.format(summed_df['total_cellnumber'].sum()))
    ground_truth_cell_numbers.append(summed_df['total_cellnumber'].sum())

The brain area wholebrain has 5156 different cell types
Total sum of neurons: 53,360,742
The brain area Cerebral cortex has 1800 different cell types
Total sum of neurons: 33,502,874
The brain area cerebellum has 1116 different cell types
Total sum of neurons: 3,939,848


In [27]:
#WE can store the values in a variable and get the scaling ratios
mfish_ctx = ground_truth_cell_numbers[1]
mfish_crb = ground_truth_cell_numbers[2]
mfish_rest = ground_truth_cell_numbers[0] - mfish_ctx - mfish_crb

In [28]:
#Scaling factors as percentage
1/(mfish_ctx/total_ctx)*100, 1/(mfish_crb/total_crb)*100, 1/(mfish_rest/total_rest)*100

(37.66369430556719, 1061.5918036629878, 103.31988184651408)

In [29]:
mfish_ctx, total_ctx, 1/(mfish_ctx/total_ctx), mfish_ctx*(1/(mfish_ctx/total_ctx))

(33502873.875372417, 12618420.0, 0.3766369430556719, 12618420.0)

In [30]:
#Sanity check
#no CTX or CRB keys should be printed. Only regions with multiple solutions, which is taken care of elsewhere

ctx_keys = []
crb_keys = []
else_keys = []
for key in combined_result_dataframes.keys():
    cluster_as_filename = parcellation_annotation[parcellation_annotation['cluster_as_filename'] == key]
    cluster = cluster_as_filename[cluster_as_filename['parcellation_term_set_name'] == 'substructure']
    #This != part is for ambiguous regions:
    if cluster.shape[0] != 1:
        print( key, cluster['parcellation_term_name'].values[0] )
        for key_name in cluster['cluster_as_filename'].values:
            # Add the element to else_keys list
            else_keys.append(key_name)
    elif cluster['label_numbers'].iloc[0] in cerebral_cortex:
        ctx_keys.append(cluster['cluster_as_filename'].iloc[0])
    elif cluster['label_numbers'].iloc[0] in cerebellum:
        crb_keys.append(cluster['cluster_as_filename'].iloc[0])
    else:
        else_keys.append(cluster['cluster_as_filename'].iloc[0])

#remove duplicates if there's any (in else there is)
else_keys = list(dict.fromkeys(else_keys))
crb_keys = list(dict.fromkeys(crb_keys))
ctx_keys = list(dict.fromkeys(ctx_keys))

print(len(ctx_keys), len(crb_keys), len(else_keys), )

fa corpus callosum, anterior forceps
icp inferior cerebellar peduncle
sV sensory root of the trigeminal nerve
scp superior cerebelar peduncles
st stria terminalis
309 54 340


Scaling: We will scale the regions one by one according to their anatomical positions:

- If the region was scaled earlier in the previous section we should not scale it anymore
- In the CTX we can scale with the cortical scaling ratio
- IN the CRB with the cerebellar scaling ratio
- Everywhere else with the residual scaling ratio
- Glia can be scaled by 1x (change)

In [31]:
mfish_ctx/total_ctx, mfish_crb/total_crb, mfish_rest/total_rest

(2.6550767746970236, 0.09419816510918158, 0.9678679283485253)

Global scaling factors from literature will be in effect:

In [32]:
%%time

#Scaling will take place
combined_result_dataframes_copy = copy.deepcopy(scaled_combined_result_dataframes)
# del scaled_combined_result_dataframes
globally_scaled_combined_result_dfs = {}

#Do the scaling: CTX: neurons only, CRB: EXC only, ELSE: neuron only
for key, df in combined_result_dataframes_copy.items():
    #Leave those regions which we have scaled already as they are
    if key in concatenated_result:
        globally_scaled_combined_result_dfs[key] = df
       
    #Scale only the neurons and glia:
    elif key in ctx_keys:
        # Select rows where the value is in neurontypes
        mask = df.index.isin(neurontypes)
        
        # Divide only selected rows by scaling factor for CTX Neuron
        df.loc[mask, 'density_mm3'] /= (mfish_ctx/total_ctx)
        globally_scaled_combined_result_dfs[key] = df
        del df, mask
        
    elif key in crb_keys:
        # Select rows where the value is in  exc types
        mask = df.index.isin(exctypes)
        # Select rows where the value is in neuron+glia types (this was an attempt to see what glia numbers would be if changed)
        # combined = np.concatenate((exctypes, glia))
        # mask = df.index.isin(combined)
        
        # Divide only selected rows by scaling factor for CB EXC
        df.loc[mask, 'density_mm3'] /= (mfish_crb/total_crb)
        globally_scaled_combined_result_dfs[key] = df        
        #print(df)
        del df, mask
    
    elif key in else_keys:
        # Select rows where the value is in neurontypes
        mask = df.index.isin(neurontypes)
        
        # Divide only selected rows by scaling factor for REST
        df.loc[mask, 'density_mm3'] /= (mfish_rest/total_rest)
        globally_scaled_combined_result_dfs[key] = df        
        del df, mask

    else:
        print(key, "is an Issue!")


# # Check if multiplication took place
# for key, df in combined_result_dataframes_copy.items():
#     if key in ctx_keys:
#         print(f'Original value for key {key}:')
#         print(df['density_mm3'])
        
# for key, df in scaled_combined_result_dataframes.items():
#     if key in ctx_keys:
#         print(f'Copied value for key {key}:')
#         print(df['density_mm3'])

#del combined_result_dataframes_copy

CPU times: user 556 ms, sys: 2.88 ms, total: 559 ms
Wall time: 557 ms


# Additional scaling factors to compensate local overwrite

In [33]:
%%time

#These numbers come from checking total values after transplant and loop scaling!
sc_factor_glia = 0.75
sc_factor_inh = 1.9
sc_factor_exc = 0.75

#Scaling will take place
combined_result_dataframes_copy = copy.deepcopy(globally_scaled_combined_result_dfs)
# del scaled_combined_result_dataframes
globally_scaled_combined_result_dfs_2 = {}

#Do the scaling:
for key, df in combined_result_dataframes_copy.items():
    #Leave those regions which we have been transplanted (local) as they are, except for GLIA:
    if key in concatenated_result:       

        # Select rows where the value is in glia
        mask = df.index.isin(glia)
        
        # Divide only selected rows by scaling factor for CTX
        df.loc[mask, 'density_mm3'] /= sc_factor_glia #Change later
        globally_scaled_combined_result_dfs_2[key] = df
        
    #Scale only in the CTX:
    elif key in ctx_keys:     
        # Select rows where the value is in glia
        mask = df.index.isin(glia)
        
        # Divide only selected rows by scaling factor for CTX GLIA
        df.loc[mask, 'density_mm3'] /= sc_factor_glia #Change later
        globally_scaled_combined_result_dfs_2[key] = df
        del mask

        # Select rows where the value is in EXC
        mask = df.index.isin(exctypes)
        
        # Divide only selected rows by scaling factor for CTX EXC
        df.loc[mask, 'density_mm3'] /= sc_factor_exc #Change later
        globally_scaled_combined_result_dfs_2[key] = df
        del mask        
        
        # Select rows where the value is in  inh types
        mask = df.index.isin(inhtypes)
        
        # Divide only selected rows by scaling factor for CTX INH
        df.loc[mask, 'density_mm3'] /= sc_factor_inh
        globally_scaled_combined_result_dfs_2[key] = df        
        #print(df)
        del df, mask

    #Scale only in the CRB:
    elif key in crb_keys:
        # Select rows where the value is in glia
        mask = df.index.isin(glia)
        
        # Divide only selected rows by scaling factor for CB GLIA
        df.loc[mask, 'density_mm3'] /= sc_factor_glia #Change later
        globally_scaled_combined_result_dfs_2[key] = df
        del mask

        # Select rows where the value is in EXC
        mask = df.index.isin(exctypes)
        
        # Divide only selected rows by scaling factor for CRB EXC
        df.loc[mask, 'density_mm3'] /= sc_factor_exc #Change later
        globally_scaled_combined_result_dfs_2[key] = df
        del mask   
        
        # Select rows where the value is in  inh types
        mask = df.index.isin(inhtypes)
        
        # Divide only selected rows by scaling factor for CB INH
        df.loc[mask, 'density_mm3'] /= sc_factor_inh
        globally_scaled_combined_result_dfs_2[key] = df        
        #print(df)
        del df, mask

    #Scale only in the REST of the brain:
    elif key in else_keys:
        # Select rows where the value is in glia
        mask = df.index.isin(glia)
        
        # Divide only selected rows by scaling factor for REST GLIA
        df.loc[mask, 'density_mm3'] /= sc_factor_glia #Change later
        globally_scaled_combined_result_dfs_2[key] = df
        del mask

        # Select rows where the value is in EXC
        mask = df.index.isin(exctypes)
        
        # Divide only selected rows by scaling factor for REST EXC
        df.loc[mask, 'density_mm3'] /= sc_factor_exc #Change later
        globally_scaled_combined_result_dfs_2[key] = df
        del mask           

        # Select rows where the value is in  inh types
        mask = df.index.isin(inhtypes)
        
        # Divide only selected rows by scaling factor for REST INH
        df.loc[mask, 'density_mm3'] /= sc_factor_inh
        globally_scaled_combined_result_dfs_2[key] = df        
        #print(df)
        del df, mask

    else:
        print(key, "is an Issue!")

CPU times: user 1.33 s, sys: 12.7 ms, total: 1.34 s
Wall time: 1.33 s


In [34]:
# Assuming 'globally_scaled_combined_result_dfs_2' is your dictionary of DataFrames
with open(f'{root_folder}scaled_densities_new.pickle', 'wb') as f:
    pickle.dump(globally_scaled_combined_result_dfs_2, f)

print(f"Saved here: {root_folder}")

Saved here: /gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/


In [35]:
globally_scaled_combined_result_dfs['AAA'] == globally_scaled_combined_result_dfs_2['AAA']

,density_mm3
cluster,
0023 L5/6 IT TPE-ENT Glut_1,True
0093 L4/5 IT CTX Glut_4,True
0150 L2/3 IT PIR-ENTl Glut_3,True
0205 MEA Slc17a7 Glut_2,True
0207 MEA Slc17a7 Glut_2,True
...,...
5309 Endo NN_1,True
5310 Endo NN_1,True
5311 Endo NN_1,True


In [74]:
globally_scaled_combined_result_dfs['LINGgranularlayer'] == globally_scaled_combined_result_dfs_2['LINGgranularlayer']

,density_mm3
cluster,
5186 CBX Golgi Gly-Gaba_1,False
5198 CB Granule Glut_1,False
5200 CB Granule Glut_2,False
5201 CB Granule Glut_2,False
5202 DCO UBC Glut_1,False
5204 DCO UBC Glut_1,False
5205 DCO UBC Glut_1,False
5207 Astro-CB NN_1,False
5208 Astro-NT NN_1,False


In [36]:
len(globally_scaled_combined_result_dfs_2)

703

We have to change back the df to contain cell types as keys:

In [52]:
def reconstruct_original_dataframes(combined_dataframes):
    """
    Reconstructs the original dictionary of dataframes (result_dataframes)
    from the transformed combined_dataframes.

    Parameters:
    - combined_dataframes: dict, where keys are clusters and values are dataframes
      of brain region densities.

    Returns:
    - result_dataframes: dict of dataframes where keys are brain region prefixes
      and values are dataframes of cell-type densities.
    """
    result_dataframes = {}

    # Iterate over each cluster and its associated dataframe
    for cluster, df in combined_dataframes.items():
        # For each row in the dataframe, assign the data to the correct prefix
        for prefix, row in df.iterrows():
            if prefix not in result_dataframes:
                result_dataframes[prefix] = pd.DataFrame(columns=df.columns)

            # Add the row to the corresponding prefix dataframe
            result_dataframes[prefix].loc[cluster] = row

    return result_dataframes

celltypes_as_keys = reconstruct_original_dataframes(globally_scaled_combined_result_dfs_2)

In [54]:
%%time

#from multiprocessing import Pool

def process_type(types, file_name):
    # Filter DataFrames based on types
    filtered_dataframes = {key: value for key, value in celltypes_as_keys.items() if key in types}
    
    # Combine filtered DataFrames
    combined_df = pd.concat(filtered_dataframes.values())
    
    # Sum the combined DataFrame by index
    summed_df = combined_df.groupby(combined_df.index).sum()
    
    # Validate result
    result = nrrd_for_validation(summed_df, parcellation_annotation, CCFv3)
    
    # Clean up
    del combined_df, summed_df, filtered_dataframes

    return (result, file_name)

def main():
    # Define the parameters for each process
    tasks = [
        (neurontypes, "scaled_total_neuron_densities"),
        (exctypes, "scaled_total_excitatory_densities"),
        (inhtypes, "scaled_total_inhibitory_densities"),
        (astrotypes, "scaled_total_astrotypes_densities"),
        (microglia, "scaled_total_microglia_densities"),
        (oligos, "scaled_total_oligocyte_densities"),
        (glia, "scaled_total_glia_densities"),
        (exci_inhib_sum, "scaled_total_excinh_densities"),
        (celltypes, "scaled_total_celltypes_densities"),
        (nonneurontypes, "scaled_total_nonneuron_densities")
    ]
    
    # Create a multiprocessing Pool
    with mp.Pool(processes=mp.cpu_count()) as pool:
        results = pool.starmap(process_type, tasks)
    
    # Sequentially write the .nrrd files to avoid concurrent writes
    for result, file_name in results:
        nrrd.write(f"{root_folder}{file_name}.nrrd", result)
        print(f"{root_folder}{file_name}.nrrd")
        
if __name__ == "__main__":
    main()

/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/scaled_total_neuron_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/scaled_total_excitatory_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/scaled_total_inhibitory_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/scaled_total_astrotypes_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/scaled_total_microglia_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/scaled_total_oligocyte_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/scaled_total_glia_densities.nrrd
/gpfs/bbp.cscs.ch/data/project/proj84/csaba/ai

# Test results

In [41]:
root_folder3

'/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'

In [49]:
file = os.path.join( root_folder, 'scaled_densities_local.pickle' )

#Load region id volumes from volume_calc_from_template.ipynb
with open(file, 'rb') as pickle_file:
    test = pickle.load(pickle_file)
print("Loaded pickle file.")

root_folder3 = '/gpfs/bbp.cscs.ch/data/project/proj84/csaba/aibs_10x_mouse_wholebrain/results/density_calculations/'
file = os.path.join( root_folder3, 'scaled_densities_new.pickle' )

#Load region id volumes from volume_calc_from_template.ipynb
with open(file, 'rb') as pickle_file:
    scaled_densities_new = pickle.load(pickle_file)
print("Loaded pickle file.")

Loaded pickle file.
Loaded pickle file.


In [50]:
len(scaled_densities_new), len(test)

(703, 703)

In [51]:
# Get the keys of both dictionaries as sets
keys_scaled_densities_new = set(scaled_densities_new.keys())
keys_test = set(test.keys())

# Find keys unique to each dictionary
unique_to_scaled_densities_new = keys_scaled_densities_new - keys_test
unique_to_test = keys_test - keys_scaled_densities_new

# Print the unique keys in both dictionaries
print("Keys unique to scaled_densities_new:", unique_to_scaled_densities_new)
print("Keys unique to test:", unique_to_test)


Keys unique to scaled_densities_new: set()
Keys unique to test: set()


In [52]:
scaled_densities_new["y"] == test["y"]

,density_mm3
cluster,
4196 NLL-SOC Spp1 Glut_2,False
4198 PG-TRN-LRN Fat2 Glut_1,False
4482 MV-SPIV Phox2b Ebf3 Lbx1 Glut_1,False
4505 MV-SPIV Zic4 Neurod2 Glut_3,False
4516 MV-SPIV Zic4 Neurod2 Glut_5,False
4521 CBN Neurod2 Pvalb Glut_1,False
4522 CBN Neurod2 Pvalb Glut_1,False
4812 POR Spp1 Gly-Gaba_1,False
4899 MY Lhx1 Gly-Gaba_3,False
